<a href="https://colab.research.google.com/github/ayushi-3536/inFairness/blob/main/compas_recidivism_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install inFairness requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 664 kB 3.3 MB/s 
     |████████████████████████████████| 21.4 MB 1.4 MB/s 
     |████████████████████████████████| 750.6 MB 11 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.


In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from inFairness.fairalgo import SenSeI
from inFairness import distances
from inFairness.auditor import SenSRAuditor, SenSeIAuditor

%load_ext autoreload
%autoreload 2

import data
import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!git clone https://github.com/ayushi-3536/inFairness.git

Cloning into 'inFairness'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 325 (delta 69), reused 35 (delta 35), pack-reused 215
Receiving objects: 100% (325/325), 1.44 MiB | 5.24 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [4]:
cd inFairness

/content/inFairness


In [5]:
!pip install -e .


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/inFairness
  Attempting uninstall: inFairness
    Found existing installation: inFairness 0.2.2
    Uninstalling inFairness-0.2.2:
      Successfully uninstalled inFairness-0.2.2
  Running setup.py develop for inFairness


In [12]:
!pwd 

/content/inFairness


In [13]:
cd examples/adult-income-prediction/

/content/inFairness/examples/adult-income-prediction


In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from inFairness.fairalgo import SenSeI
from inFairness import distances
from inFairness.auditor import SenSRAuditor, SenSeIAuditor

%load_ext autoreload
%autoreload 2

import data
import metrics


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
class AdultDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        return data, label
    
    def __len__(self):
        return len(self.labels)



In [81]:
train_df, test_df = data.load_data()

X_train_df, Y_train_df = train_df
X_test_df, Y_test_df = test_df

# Create test data with spouse variable flipped
X_test_df_spouse_flipped = X_test_df.copy()
X_test_df_spouse_flipped.relationship_Wife = 1 - X_test_df_spouse_flipped.relationship_Wife

X_train_df.head()

,age,capital-gain,capital-loss,education-num,hours-per-week,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,...,relationship_Unmarried,relationship_Wife,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.409331,-0.14652,-0.218253,-1.613806,-0.496770,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
1,-1.104187,-0.14652,-0.218253,-0.050064,-1.741764,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1.393118,-0.14652,-0.218253,-0.440999,2.574214,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
3,-0.423104,-0.14652,-0.218253,-0.440999,1.163221,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,-0.877159,-0.14652,-0.218253,1.122743,0.748224,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


In [75]:
device = torch.device('cpu')

protected_vars = ['race_White', 'sex_Male']
protected_idxs = [X_train_df.columns.get_loc(var) for var in protected_vars]

X_train, y_train = data.convert_df_to_tensor(X_train_df, Y_train_df)
X_test, y_test = data.convert_df_to_tensor(X_test_df, Y_test_df)
X_test_flip, y_test_flip = data.convert_df_to_tensor(X_test_df_spouse_flipped, Y_test_df)



In [ ]:
# Create the training and testing dataset
train_ds = AdultDataset(X_train, y_train)
test_ds = AdultDataset(X_test, y_test)
test_ds_flip = AdultDataset(X_test_flip, y_test_flip)

# Create train and test dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1000, shuffle=False)
test_dl_flip = DataLoader(test_ds_flip, batch_size=1000, shuffle=False)


In [52]:
# Create a fully connected neural network

class Model(nn.Module):

    def __init__(self, input_size, output_size):

        super().__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fcout = nn.Linear(100, output_size)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fcout(x)
        return x


In [53]:
### Standard training

input_size = X_train.shape[1]
output_size = 2

network_standard = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_standard.parameters(), lr=1e-3)
loss_fn = F.cross_entropy

EPOCHS = 10


In [54]:
network_standard.train()

for epoch in range(EPOCHS):

    for x, y in train_dl:

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = network_standard(x).squeeze()
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'{epoch} completed')

0 completed


In [55]:
accuracy = metrics.accuracy(network_standard, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_standard, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_standard, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')
print(f'Spouse consistency: {spouse_consistency}')

Accuracy: 0.8522777557373047
Balanced accuracy: 0.7630170766626202
Spouse consistency: 0.933547103051747


In [83]:
### Individually fair training

network_fair = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair.parameters(), lr=1e-3)
lossfn = F.cross_entropy

distance_x = distances.SVDSensitiveSubspaceDistance()
distance_y = distances.SquaredEuclideanDistance()


In [88]:
# distance_x.fit(X_train, protected_idxs)
distance_x.fit(X_train, n_components=10)

distance_y.fit(num_dims=output_size)

distance_x.to(device)
distance_y.to(device)

In [89]:
rho = 5.0
eps = 0.1
auditor_nsteps = 100
auditor_lr = 1e-3

fairalgo = SenSeI(network_fair, distance_x, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)


In [90]:
fairalgo.train()

for epoch in range(EPOCHS):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo(x, y)
        result.loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'{epoch} completed')

0 completed


In [91]:
accuracy = metrics.accuracy(network_fair, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_fair, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_fair, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')
print(f'Spouse consistency: {spouse_consistency}')

Accuracy: 0.8374612927436829
Balanced accuracy: 0.7381569325516283
Spouse consistency: 1.0


In [93]:
# Auditing using the SenSR Auditor

audit_nsteps = 1000
audit_lr = 0.1

auditor = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in true_divide


Loss ratio (Standard model) : 2.166024779650726. Is model fair: False
Loss ratio (fair model) : 1.061765205480569. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [97]:
# Auditing using the SenSeI Auditor

audit_nsteps = 500
audit_lr = 0.001

auditor = SenSeIAuditor(distance_x=distance_x, distance_y=distance_y, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)


Loss ratio (Standard model) : 65720251.76228812. Is model fair: False
Loss ratio (fair model) : 1.3731180167326456. Is model fair: False
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [113]:
import os
import requests
import zipfile

bank_full_data= 'https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip'
    
dirpaths = {}
rootdir = "./bank_marketing_dataset"
os.makedirs(rootdir, exist_ok=True)

import requests, zipfile, io
r = requests.get(bank_full_data)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(rootdir)

# r = requests.get(bank_full_data)
# with zipfile.ZipFile(bank_full_data, 'r') as zip_ref:
#     zip_ref.extractall(rootdir)

    

In [170]:
def get_input_output_df(data,target):

    cols = sorted(data.columns)
    output_col = target
    input_cols = [col for col in cols if col not in output_col]

    df_X = data[input_cols]
    df_Y = data[output_col]
    
    return df_X, df_Y

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
names = ['age','job','marital','education',
    'default','balance','housing','loan','contact',
    'day','month','duration','campaign','pdays',
    'previous','poutcome','y']
data = pd.read_csv(
            rootdir+'/bank-full.csv', sep=';', header=None, names=names,
            na_values=['?'], skipinitialspace=True,skiprows=1
        )

# remove rows with NaNs
data.dropna(inplace=True)
print(data)
categorical_vars = [
        'job', 'marital', 'education', 
        'default', 'housing', 'contact', 'month',
        'poutcome'
    ]
data = pd.get_dummies(data, columns=categorical_vars)
print(data)

# Split into train/test splits
train_data = data.sample(frac=0.8, random_state=123)
test_data = data.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

continuous_vars = [
        'age', 'balance', 'day', 'duration',
        'campaign', 'pdays','previous'
    ]
scaler = StandardScaler().fit(train_data[continuous_vars])
train_data[continuous_vars] = scaler.transform(train_data[continuous_vars])
test_data[continuous_vars] = scaler.transform(test_data[continuous_vars])

train_data = get_input_output_df(train_data,'y')
test_data = get_input_output_df(test_data,'y')


In [129]:
X_train_df, Y_train_df = train_data
X_test_df, Y_test_df = test_data


In [130]:
X_train_df.head()


,age,balance,campaign,contact_cellular,contact_telephone,contact_unknown,day,default_no,default_yes,duration,...,month_may,month_nov,month_oct,month_sep,pdays,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,previous
0,1.410405,-0.248010,-0.246580,0,0,1,1.582194,1,0,1.078641,...,1,0,0,0,-0.411002,0,0,0,1,-0.242557
1,-0.371716,-0.228508,-0.246580,1,0,0,-1.058473,1,0,-0.755420,...,0,0,0,0,-0.411002,0,0,0,1,-0.242557
2,-1.309675,0.201506,-0.568108,1,0,0,-1.178503,1,0,-0.626919,...,0,0,0,0,-0.411002,0,0,0,1,-0.242557
3,0.191059,-0.439127,0.396476,1,0,0,0.741982,1,0,-0.073975,...,0,0,0,0,-0.411002,0,0,0,1,-0.242557
4,1.597997,-0.702076,0.396476,1,0,0,-0.938443,1,0,-0.712586,...,1,0,0,0,-0.411002,0,0,0,1,-0.242557


In [ ]:

protected_vars = ['marital']
protected_idxs = [X_train_df.columns.get_loc(var) for var in protected_vars]

X_train, y_train = data.convert_df_to_tensor(X_train_df, Y_train_df)
X_test, y_test = data.convert_df_to_tensor(X_test_df, Y_test_df)

In [131]:
train_ds = AdultDataset(X_train, y_train)
test_ds = AdultDataset(X_test, y_test)
test_ds_flip = AdultDataset(X_test_flip, y_test_flip)

# Create train and test dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1000, shuffle=False)
test_dl_flip = DataLoader(test_ds_flip, batch_size=1000, shuffle=False)


In [182]:
def training(X_train,train_dl,test_dl):
  input_size = X_train.shape[1]
  output_size = 2
  network_standard = Model(input_size, output_size).to(device)
  optimizer = torch.optim.Adam(network_standard.parameters(), lr=1e-3)
  loss_fn = F.cross_entropy

  EPOCHS = 10
  network_standard.train()

  for epoch in range(EPOCHS):

      for x, y in train_dl:

          x, y = x.to(device), y.to(device)
          optimizer.zero_grad()
          y_pred = network_standard(x).squeeze()
          loss = loss_fn(y_pred, y)
          loss.backward()
          optimizer.step()

      if epoch % 10 == 0:
          print(f'{epoch} completed')
  accuracy = metrics.accuracy(network_standard, test_dl, device)
  balanced_acc = metrics.balanced_accuracy(network_standard, test_dl, device)
  print(f'Accuracy: {accuracy}')
  print(f'Balanced accuracy: {balanced_acc}')

In [187]:


### Individually fair training

network_fair = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair.parameters(), lr=1e-3)
lossfn = F.cross_entropy

distance_x = distances.SVDSensitiveSubspaceDistance()
distance_y = distances.SquaredEuclideanDistance()

distance_x.fit(X_train, n_components=10)
distance_y.fit(num_dims=output_size)

distance_x.to(device)
distance_y.to(device)

rho = 5.0
eps = 0.1
auditor_nsteps = 100
auditor_lr = 1e-3

fairalgo = SenSeI(network_fair, distance_x, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)

fairalgo.train()

for epoch in range(EPOCHS):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo(x, y)
        result.loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'{epoch} completed')

accuracy = metrics.accuracy(network_fair, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_fair, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_fair, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')

In [137]:
# Auditing using the SenSR Auditor

audit_nsteps = 1000
audit_lr = 0.1

auditor = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)





/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in true_divide


Loss ratio (Standard model) : 3.0158437912788316. Is model fair: False
Loss ratio (fair model) : 1.09732222633876. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [138]:
# Auditing using the SenSeI Auditor

audit_nsteps = 500
audit_lr = 0.001

auditor = SenSeIAuditor(distance_x=distance_x, distance_y=distance_y, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: divide by zero encountered in true_divide


Loss ratio (Standard model) : 7323198008023.832. Is model fair: False
Loss ratio (fair model) : 1.3744366240955816. Is model fair: False
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [179]:
import os
import requests
import zipfile
compas= 'https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv'
data_df = pd.read_csv(compas,index_col=0,parse_dates=[0])
cols_to_drop = [
        'name','first','last','compas_screening_date','dob','age_cat',
        'v_screening_date','in_custody','out_custody','start','end',
        'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date',
       'c_arrest_date', 'c_days_from_compas',
       'c_charge_desc', 'is_recid', 'r_case_number', 
       'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in',
       'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'screening_date','days_b_screening_arrest',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text','event'
    ]

data_df.drop(cols_to_drop, axis=1, inplace=True)


categorical_vars = [
        'sex', 'race', 'c_charge_degree','r_charge_degree', 
        'type_of_assessment', 'score_text'
    ]
data_df = pd.get_dummies(data_df, columns=categorical_vars)
print(data_df.columns)
cols_to_drop=['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Native American',
       'race_Other','sex_Female']

data_df.drop(cols_to_drop, axis=1, inplace=True)
train_data = data_df.sample(frac=0.8, random_state=123)
test_data = data_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
continuous_vars = [
    'juv_fel_count', 'decile_score', 'juv_misd_count',
    'juv_other_count', 'priors_count','decile_score.1', 
    'priors_count.1'
    ]
scaler = StandardScaler().fit(train_data[continuous_vars])
train_data[continuous_vars] = scaler.transform(train_data[continuous_vars])
test_data[continuous_vars] = scaler.transform(test_data[continuous_vars])

train_data = get_input_output_df(train_data,'two_year_recid')
test_data = get_input_output_df(test_data,'two_year_recid')

X_train_df, Y_train_df = train_data
X_test_df, Y_test_df = test_data

X_train_df.head



Index(['age', 'juv_fel_count', 'decile_score', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'decile_score.1', 'priors_count.1',
       'two_year_recid', 'sex_Female', 'sex_Male', 'race_African-American',
       'race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Native American',
       'race_Other', 'c_charge_degree_F', 'c_charge_degree_M',
       'r_charge_degree_(CO3)', 'r_charge_degree_(F1)', 'r_charge_degree_(F2)',
       'r_charge_degree_(F3)', 'r_charge_degree_(F5)', 'r_charge_degree_(F6)',
       'r_charge_degree_(F7)', 'r_charge_degree_(M1)', 'r_charge_degree_(M2)',
       'r_charge_degree_(MO3)', 'type_of_assessment_Risk of Recidivism',
       'score_text_High', 'score_text_Low', 'score_text_Medium'],
      dtype='object')


<bound method NDFrame.head of       age  c_charge_degree_F  c_charge_degree_M  decile_score  decile_score.1  \
0      28                  1                  0     -0.873321       -0.873321   
1      22                  1                  0      1.925477        1.925477   
2      44                  0                  1     -1.223171       -1.223171   
3      29                  0                  1     -0.873321       -0.873321   
4      29                  1                  0     -0.173622       -0.173622   
...   ...                ...                ...           ...             ...   
5766   48                  1                  0      0.875928        0.875928   
5767   26                  0                  1      0.875928        0.875928   
5768   51                  1                  0     -1.223171       -1.223171   
5769   27                  1                  0     -0.523471       -0.523471   
5770   32                  1                  0      0.875928        0.875928  

In [184]:
import data
protected_vars = ['sex_Male', 'race_African-American']
protected_idxs = [X_train_df.columns.get_loc(var) for var in protected_vars]

X_train, y_train = data.convert_df_to_tensor(X_train_df, Y_train_df)
X_test, y_test = data.convert_df_to_tensor(X_test_df, Y_test_df)
X_train_df, Y_train_df = train_data
X_test_df, Y_test_df = test_data
train_ds = AdultDataset(X_train, y_train)
test_ds = AdultDataset(X_test, y_test)
# Create train and test dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1000, shuffle=False)


0 completed
Accuracy: 0.9708939790725708
Balanced accuracy: 0.972931198478123


In [191]:
input_size = X_train.shape[1]
output_size = 2
network_standard = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_standard.parameters(), lr=1e-3)
loss_fn = F.cross_entropy

EPOCHS = 10
network_standard.train()
for epoch in range(EPOCHS):

      for x, y in train_dl:

          x, y = x.to(device), y.to(device)
          optimizer.zero_grad()
          y_pred = network_standard(x).squeeze()
          loss = loss_fn(y_pred, y)
          loss.backward()
          optimizer.step()

      if epoch % 10 == 0:
          print(f'{epoch} completed')
accuracy = metrics.accuracy(network_standard, test_dl, device)
print(f'Accuracy: {accuracy}')

0 completed
Accuracy: 0.9708939790725708


In [192]:

### Individually fair training

network_fair = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair.parameters(), lr=1e-3)
lossfn = F.cross_entropy

distance_x = distances.SVDSensitiveSubspaceDistance()
distance_y = distances.SquaredEuclideanDistance()

distance_x.fit(X_train, n_components=10)
distance_y.fit(num_dims=output_size)

distance_x.to(device)
distance_y.to(device)

rho = 5.0
eps = 0.1
auditor_nsteps = 100
auditor_lr = 1e-3

fairalgo = SenSeI(network_fair, distance_x, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)

fairalgo.train()

for epoch in range(EPOCHS):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo(x, y)
        result.loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'{epoch} completed')

accuracy = metrics.accuracy(network_fair, test_dl, device)

print(f'Accuracy: {accuracy}')

0 completed
Accuracy: 0.9085239171981812


In [193]:
# Auditing using the SenSR Auditor

audit_nsteps = 1000
audit_lr = 0.1

auditor = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in true_divide


Loss ratio (Standard model) : 542.5004962436053. Is model fair: False
Loss ratio (fair model) : 1.0210911146684414. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [194]:
# Auditing using the SenSeI Auditor

audit_nsteps = 500
audit_lr = 0.001

auditor = SenSeIAuditor(distance_x=distance_x, distance_y=distance_y, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: divide by zero encountered in true_divide


Loss ratio (Standard model) : 135575.64954199982. Is model fair: False
Loss ratio (fair model) : 1.0012931291001776. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model
